In [1]:
import numpy as np

from lib.read_data import cycles, u_desvio as u, y_desvio as y

model_name = "SS"


In [2]:
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from typing import Callable

u_funcs = [interp1d(cycles, u[:, i], kind="previous", fill_value="extrapolate") for i in range(u.shape[1])]

def linear_diff_eq(
    t: float, x: np.ndarray, u_funcs: list[Callable[[float], float]], A: np.ndarray, B: np.ndarray
):
    u = np.array([u_func(t) for u_func in u_funcs])

    dx_dt = A @ x + B @ u
    return dx_dt

X0 = -np.ones(3*3+3*4)

def objetivo(X0):
    A = X0[:9].reshape((3, 3))
    B = X0[9:].reshape((3, 4))

    sol = solve_ivp(
        lambda t, x: linear_diff_eq(t, x, u_funcs, A, B),
        [cycles[0], cycles[-1]],
        y[0],
        t_eval=cycles,
        method="RK45",
    )

    return np.sum((sol.y.T - y) ** 2)


res = minimize(objetivo, X0, method="BFGS")
A_opt = res.x[:9].reshape((3, 3))
B_opt = res.x[9:].reshape((3, 4))

sol = solve_ivp(
    lambda t, x: linear_diff_eq(t, x, u_funcs, A_opt, B_opt),
    [cycles[0], cycles[-1]],
    y[0],
    t_eval=cycles,
    method="RK45",
)

y_ss = sol.y.T


In [3]:
print("Matriz A otimizada:")
print(A_opt)
print("\nMatriz B otimizada:")
print(B_opt)


Matriz A otimizada:
[[-0.99999232 -0.99999466 -0.99999486]
 [-0.99999531 -0.9999966  -0.99999671]
 [-1.00001288 -1.00000925 -1.00000893]]

Matriz B otimizada:
[[-0.99998869 -0.9991219  -0.99999996 -0.99998359]
 [-1.0000072  -1.00001527 -1.00000063 -0.99998876]
 [-1.00002133 -0.99992588 -1.00000595 -0.99908953]]


In [4]:
from lib.read_data import cycles,  y as y_true, y_ref

# Convertendo o valor desvio em variavel de engenharia
y_ss = y_ss + np.tile(y_ref, (y_ss.shape[0], 1))

# Plotando resultados
from lib.plot import plot_comparison

plot_comparison(cycles, y_true, y_ss, model_name)
